# Example usage

To use `ghgql` in a project:

I've designed `ghgql` to work on query files rather than pure query strings. This is fully intentional to not get into the habit of putting query strings all around in the code. Instead when the queries live in files we can validate them agains Github's GraphQL schema.

Long story short, instead of using `ghgql.GithubGraphQL.query()` you should use `ghgql.GithubGraphQL.query_from_file()`!

# Running our first query

Suppose we have the following query to fetch one issue from the `llvm/llvm-project` repository:

In [1]:
query = """
query ($searchQuery: String!) {
  search(query: $searchQuery, type: ISSUE, first: 1) {
    edges {
      node {
        ... on Issue {
          id
          number
          title
          url
        }
      }
    }
  }
}
"""

This is how we can use it with `ghgql`:

In [2]:
import os
import ghgql

with ghgql.GithubGraphQL(token=os.getenv("GITHUB_TOKEN")) as ghapi:
    result = ghapi.query(query=query, variables={"searchQuery": "llvm/llvm-project"})

## Easy inspection of results

The main thing that this library does apart from querying the Github GraphQL API is to help you access elements in the result of a query. Let's first look at how the `result` looks like:

In [3]:
    result

{'data': {'search': {'edges': [{'node': {'id': 'I_kwDOHicqdc5RG-tC',
      'number': 16,
      'title': 'llvm/llvm-project',
      'url': 'https://github.com/KhushP786/open-sauced-goals/issues/16'}}]}}}

Notice that result is a dictionary with a top-level `"data"` key. This indicates that there are no errors. if there were errors, we would see a top-level `errors` element.

### Convenient access

The `ghgql` library tries to make it easy to query for nested results under the nested `"data"` key by providing a `get(name, default)` method. Here is how it works: 

Let's first query for the `edges` key the data element by using the `get()` method of the `result`: 

In [4]:
    result.get("search.edges")

[{'node': {'id': 'I_kwDOHicqdc5RG-tC',
   'number': 16,
   'title': 'llvm/llvm-project',
   'url': 'https://github.com/KhushP786/open-sauced-goals/issues/16'}}]

Now, you could argue that this is not super helpful because you could have written:

In [5]:
    result["data"]["search"]["edges"]

[{'node': {'id': 'I_kwDOHicqdc5RG-tC',
   'number': 16,
   'title': 'llvm/llvm-project',
   'url': 'https://github.com/KhushP786/open-sauced-goals/issues/16'}}]

And while this is true, we're doing a bit more under the covers which we'll discuss in the next sections!

### Defaults
When you query for something that doesn't exist, you'll get at `KeyError` exception:

In [6]:
    try:
        result.get("search.non.existing.key")
    except KeyError as ex:
        print(str(ex))

'key "search.non.existing.key" is not found and default is not present'


Now, as the exception indicates, we haven't specified a `default` that the `get()` function could have used in case we want to ignore a non-existing key. Let's do that: 

In [7]:
    result.get("search.non.existing.key", default=42)

42

Notice that the exception is gone because we have specified a `default` parameter.

### Handle errors

God forbid, but there might be errors when you're writing a GraphQL query. Let's query github with a completely invalid query and inspect the results: 

In [8]:
query="Yes, I'm invalid!"
with ghgql.GithubGraphQL(token=os.getenv("GITHUB_TOKEN")) as ghapi:
    result = ghapi.query(query=query, variables={"searchQuery": "llvm/llvm-project"})
    print(result)

{'errors': [{'message': 'Parse error on "Yes" (IDENTIFIER) at [1, 1]', 'locations': [{'line': 1, 'column': 1}]}]}


Notice that ther no longer is a `data` key on the top-level of the `result` dictionary. We can expect a `RuntimeError` to be raised when querying for anything, with and without a `default`. This is because `errors` have to deal with first!

In [9]:
    try:
        result.get("search.edges")
    except RuntimeError as ex:
        print(str(ex))

errors are present


#### Result.errors and Result.data
You can of course check the result for `errors` yourself. Here are a few ways to do this:

In [10]:
    # The usual safe way to access the dict
    if "errors" in result:
        print(result["errors"])
    
    # The convenient way through a property on the Result class
    if result.errors != None:
        print(result.errors)

[{'message': 'Parse error on "Yes" (IDENTIFIER) at [1, 1]', 'locations': [{'line': 1, 'column': 1}]}]
[{'message': 'Parse error on "Yes" (IDENTIFIER) at [1, 1]', 'locations': [{'line': 1, 'column': 1}]}]


The `result.errors` is a property that you can always query even when there's no `"errors"` top-level key in the result. It returns `None` when there are no errors. The same is true for `result.data` which of course returns the `result["data"]` value or `None` if `"data"` doesn't exist as a top-level key.

# Conclusion

`ghgql` provides ways to query the Github GraphQL and allows for easy inspection of the resulting objects.

TODO(kwk): In the future we can show how mutations work with `ghgql`.